In [51]:
from transformers import AutoImageProcessor, ViTForImageClassification
from PIL import Image
import requests
import torch
import csv
import os

processor = AutoImageProcessor.from_pretrained('touchtech/fashion-images-gender-age')
model = ViTForImageClassification.from_pretrained('touchtech/fashion-images-gender-age')

In [52]:
labels = model.config.id2label
def predict_gender(file):
    image = Image.open(file)
    inputs = processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    predicted_label_index = torch.argmax(outputs.logits, dim=1).item()
    predicted_label = labels[predicted_label_index]
    gender_map = {
        "model-female-adult": "female",
    "model-female-child": "female",
    "model-male-adult": "male",
    "model-male-child": "male",
    "pack": "uncertain"
    }
    return gender_map[predicted_label]

In [55]:
dir_path = "/home/marmik/Desktop/fabric/fabric-dataset/val/cotton"
fp = open("gender_prediction.csv", "w")
writer = csv.writer(fp)
writer.writerow(["Filepath", "Gender"])
for file in os.listdir(dir_path):
    if file.lower().endswith(".jpg"):
        gender = predict_gender(os.path.join(dir_path, file))
        writer.writerow([file, gender])
fp.close()